In [ ]:
import os
import numpy as np
import pandas as pd
import sqlalchemy
from math import log
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from itertools import product
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib
from pycaret.classification import *

In [ ]:
def createKmerSet(kmersize):
    '''
    write all possible kmers
    :param kmersize: integer, 8
    :return uniq_kmers: list of sorted unique kmers
    '''
    kmerSet = set()
    nucleotides = ["a", "c", "g", "t"]    
    kmerall = product(nucleotides, repeat=kmersize)
    for i in kmerall:
        kmer = ''.join(i)
        kmerSet.add(kmer)
    uniq_kmers = sorted(list(kmerSet))  
    return uniq_kmers

def compute_kmer_entropy(kmer):
    '''
    compute shannon entropy for each kmer
    :param kmer: string
    :return entropy: float
    '''
    prob = [float(kmer.count(c)) / len(kmer) for c in dict.fromkeys(list(kmer))]
    entropy = - sum([ p * log(p) / log(2.0) for p in prob ])
    return round(entropy, 2)

def make_stopwords(kmersize):
    '''
    write filtered out kmers
    :param kmersize: integer, 8
    :return stopwords: list of sorted low-complexity kmers
    '''
    kmersize_filter = {5:1.3, 6:1.3, 7:1.3, 8:1.3, 9:1.3, 10:1.3}
    limit_entropy = kmersize_filter.get(kmersize)
    kmerSet = set()
    nucleotides = ["a", "c", "g", "t"]    
    kmerall = product(nucleotides, repeat=kmersize)
    for n in kmerall:
        kmer = ''.join(n)
        if compute_kmer_entropy(kmer) < limit_entropy:
            kmerSet.add(make_newtoken(kmer))
        else:
            continue
    stopwords = sorted(list(kmerSet))
    return stopwords

def createNewtokenSet(kmersize):
    '''
    write all possible newtokens
    :param kmersize: integer, 8
    :return uniq_newtokens: list of sorted unique newtokens
    ''' 
    newtokenSet = set()
    uniq_kmers = createKmerSet(kmersize)
    for kmer in uniq_kmers:
        newtoken = make_newtoken(kmer)
        newtokenSet.add(newtoken)  
    uniq_newtokens = sorted(list(newtokenSet))
    return uniq_newtokens      


def make_newtoken(kmer):
    '''
    write a collapsed kmer and kmer reverse complementary as a newtoken
    :param kmer: string e.g., "AT"
    :return newtoken: string e.g., "atnta"
    :param kmer: string e.g., "TA"
    :return newtoken: string e.g., "atnta"
    '''
    kmer = str(kmer).lower()
    newtoken = "n".join(sorted([kmer,kmer.translate(str.maketrans('tagc', 'atcg'))[::-1]]))
    return newtoken

def write_ngrams(sequence):
    '''
    write a bag of newtokens of size n
    :param sequence: string e.g., "ATCG"
    :param (intern) kmerlength e.g., 2
    :return newtoken_string: string e.g., "atnta" "gatc" "cgcg" 
    '''
    seq = str(sequence).lower()
    finalstart = (len(seq)-kmerlength)+1
    allkmers = [seq[start:(start+kmerlength)] for start in range(0,finalstart)]
    tokens = [make_newtoken(kmer) for kmer in allkmers if len(kmer) == kmerlength and "n" not in kmer]
    newtoken_string = " ".join(tokens)
    return newtoken_string


In [ ]:
'''
An example for G1 data in Arabidopsis
'''

# Set the parameters of the model
kmerlength=7
newtoken_size = 1+(kmerlength*2)
all_tokens = createNewtokenSet(kmerlength)
full=False
stpwrds = make_stopwords(kmerlength)
expected_tokens = len(all_tokens)

# Import information of ChIP peaks and ATAC peaks in 5 species as train data sets.

df=pd.read_table('kmer_AT_G1', sep="\t",
              header=0)
df_T = df[df.bound == 1].reset_index(drop=True)
df_F = df[df.bound == 0].reset_index(drop=True)


# Filter n groups of non-overlapping negative examples with the same number of positive examples, and build a list.
df_F_list = []
F_num = len(df_F)
T_num = len(df_T)
for i in range(F_num // T_num):
    df_F_i = df_F.sample(T_num, random_state=888)
    df_F_list.append(df_F_i.reset_index(drop=True))
    # Update df_F, delete the part that has been taken
    index_no_i = [i for i in df_F.index if i not in df_F_i.index]
    df_F = df_F.loc[index_no_i].reset_index(drop=True)

tmpvectorizer = TfidfVectorizer(min_df = 1 , max_df = 1.0, sublinear_tf=True,use_idf=True)
X_TFIDF_ALL =  tmpvectorizer.fit_transform(all_tokens) #newtoken sequences to numeric index.
vcblry = tmpvectorizer.get_feature_names()

if full:
    print("keeping all low-complexity k-mers")
    kmer_names = vcblry
    feature_names = np.asarray(kmer_names) #key transformation to use the fancy index into the report
else:
    print("removing %d low-complexity k-mers" % len(stpwrds))
    kmer_names = [x for x in vcblry if x not in stpwrds]
    feature_names = np.asarray(kmer_names) #key transformation to use the fancy index into the report

vectorizer = TfidfVectorizer(min_df = 1 , max_df = 1.0, sublinear_tf=True,use_idf=True,vocabulary=kmer_names)

tmpvectorizer = TfidfVectorizer(min_df=1,
                                max_df=1.0,
                                sublinear_tf=True,
                                use_idf=True)
X_TFIDF_ALL = tmpvectorizer.fit_transform(
    all_tokens)  #newtoken sequences to numeric index.
vcblry = tmpvectorizer.get_feature_names()
def get_all_model(df_T=df_T, df_F_list=df_F_list):
    k = 0
    model_list = []
    LR_hold_TFIDF_pred_list = []
    Y_holdout_list = []
    X_TFIDF_test_list = []
    # Iteratively build a balanced dataset, build a model, train, save predictions.
    for i in df_F_list:
        df_balance_i = df_T.append(i).reset_index(drop=True)
        train_i, test_i = train_test_split(df_balance_i,
                                           test_size=0.2,
                                           random_state=333,
                                           shuffle=True)
        train_i = train_i.reset_index(drop=True)
        test_i = test_i.reset_index(drop=True)
        train_i["tokens"] = train_i["dna_string"].apply(write_ngrams)
        test_i["tokens"] = test_i["dna_string"].apply(write_ngrams)
        train_tokens_i = train_i["tokens"].tolist()
        test_tokens_i = test_i["tokens"].tolist()
        train_labels_i = train_i["bound"].tolist()
        test_labels_i = test_i["bound"].tolist()
        unique_train_labels = len(list(set(train_labels_i)))
        unique_test_labels = len(list(set(test_labels_i)))
        Y_DEV_i = np.asarray(train_labels_i)
        Y_holdout_i = np.asarray(test_labels_i)
        X_TFIDF_DEV_i = vectorizer.fit_transform(train_tokens_i)
        X_TFIDF_test_i = vectorizer.fit_transform(test_tokens_i)
        locals()['TFIDF_LR_' + str(k)] = LogisticRegression(
            C=1.0,
            class_weight=None,
            dual=False,
            fit_intercept=True,
            intercept_scaling=1,
            max_iter=100,
            multi_class='ovr',
            n_jobs=1,
            penalty='l2',
            random_state=None,
            solver='liblinear',
            tol=0.0001,
            verbose=0,
            warm_start=False)
        locals()['TFIDF_LR_' + str(k)].fit(X_TFIDF_DEV_i, Y_DEV_i)
        model_list.append(locals()['TFIDF_LR_' + str(k)])
        LR_hold_TFIDF_pred_list.append(
            locals()['TFIDF_LR_' + str(k)].predict(X_TFIDF_test_i))
        Y_holdout_list.extend(Y_holdout_i)
        X_TFIDF_test_list.append(X_TFIDF_test_i)
        k = k + 1
    return model_list, LR_hold_TFIDF_pred_list, Y_holdout_list, X_TFIDF_test_list

model_all, hold_TFIDF_pred_all, Y_holdout_all, X_TFIDF_test_all = get_all_model()



# Export the kmer weights from the LR classifier to a sqlite3 database
if hasattr(model_all[0], 'coef_'):
    top = np.argsort(model_all[0].coef_[0])[-5:] #select the top 5 index
    botton = np.argsort(model_all[0].coef_[0])[:5] #select the bottom 5 index
#    logging.info("database table LR_results")
#    logging.info("top 5 positive kmers")
#    logging.info(" ".join([ i.split('n')[0].upper() for i in feature_names[top] ]))
#    logging.info(" ".join([ i.split('n')[1].upper() for i in feature_names[top] ]))
#    logging.info("top 5 negative kmers")
#    logging.info(" ".join([ i.split('n')[0].upper() for i in feature_names[botton] ]))
#    logging.info(" ".join([ i.split('n')[1].upper() for i in feature_names[botton] ]))
    print("Saving data to database table LR_results")
    print('*' * 80)
    print("%s: %s" % ("pos kmers", " ".join([ i.split('n')[0].upper() for i in feature_names[top] ]) ))
    print("%s: %s" % ("pos kmers", " ".join([ i.split('n')[1].upper() for i in feature_names[top] ]) ))
    print() #making room
    print("%s: %s" % ("neg kmers", " ".join([ i.split('n')[0] for i in feature_names[botton] ]) ))
    print("%s: %s" % ("neg kmers", " ".join([ i.split('n')[1] for i in feature_names[botton] ]) ))
    print('*' * 80)
    print() #making room
    LR_weights = []
    for idx, kmer_score in enumerate(model_all[0].coef_[0]):
        features = feature_names[idx].split('n')
        LR_weights.append({'kmer':features[0].upper(),'revcomp':features[1].upper(),'score':kmer_score})
    LR_weights_feature = pd.DataFrame(LR_weights)
    with open("AT_G1_kmer_result.tsv","w") as f:
        f.writelines('kmer'+'\t'+'revcomp'+'\t'+'score'+'\n')
        for n in range(LR_weights_feature.shape[0]):
            f.writelines(LR_weights_feature.kmer[n]+'\t'+LR_weights_feature.revcomp[n]+'\t'+str(LR_weights_feature.score[n])+'\n')



In [ ]:
# Ensemble model
proba_list = []
for i in range(len(model_all)):
    model_list = []
    for j in range(len(model_all)):
        model_list.extend(model_all[i].predict_proba(X_TFIDF_test_all[j])[:, 1])
    proba_list.append(model_list)
proba_ = np.array(proba_list).mean(axis=0)
Y_pre = [0 if i < 0.5 else 1 for i in proba_]
print('Ensemble model: ')
print(metrics.classification_report(Y_holdout_all, Y_pre))

# Draw ROC curve
fpr,tpr,threshold = metrics.roc_curve(Y_holdout_all,proba_)
roc_auc = metrics.auc(fpr,tpr)
plt.stackplot(fpr,tpr,color= 'steelblue',alpha=0.3,edgecolor='black')
plt.plot(fpr,tpr,color='black',lw=1)
plt.plot([0,1],[0,1],color='red',linestyle='--')
plt.text(0.5,0.3,'ROC curve (area = %.2f)' % roc_auc)
plt.xlabel("1-Specificity")
plt.ylabel('Sensitivity')
plt.savefig('at_g1_kmer_roc.pdf')